---
title: "2023_Kronensicherung_Plesse_Analyse"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Analyse

## Kombinierte Analyse LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten

### IMPORT: Packages

In [12]:
# Struktur & Typen
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Zum Vereinheitlichen von Strings

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

# Statistik
from scipy.stats import linregress, f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as mc

In [13]:
# Eigene Module und Funktionen
from kj_core.utils.latex_export import (
    generate_latex_table,
    generate_grouped_latex_tables,
    save_latex_table,
    build_data_dict_df
)
from kj_core.utils.labeling import (
    get_label_from_dict,
    get_color_dict
)
from kj_core import (
    CoreConfig,
    PlotManager,
    get_logger
)

# Projekteinstellungen
from project_config import (
    working_directory,
    data_export_directory,
    latex_export_directory,
    filename_clean_dataset,
    filename_clean_data_dict
)

### IMPORT: Manager Instanzen

In [14]:
CONFIG = CoreConfig(working_directory=f"{working_directory}/combined")
PLOT_MANAGER = PlotManager(CONFIG)
logger = get_logger(__name__)

2025-04-08 10:15:32 [WARNING] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined already exists.
2025-04-08 10:15:32 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined!
2025-04-08 10:15:32 [INFO] kj_core.core_config.__init__: <CoreConfig>
  Package:            kj_core
  Working Directory:  C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined
  Plot Directory:     C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined\plots
  Data Directory:     C:\kyellsen\005_Projekte\2024

## IMPORT: Daten Import

In [15]:
# Dateien laden
df = pd.read_feather(data_export_directory / filename_clean_dataset)

with open(data_export_directory / filename_clean_data_dict, "r", encoding="utf-8") as f:
    data_dict = json.load(f)

## ANALYSE: Explorative Datenanalyse

In [16]:
# In DataFrame umwandeln
data_dict_df= build_data_dict_df(data_dict)

# In Markdown umwandeln und anzeigen
md_text = data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable                | Kategorie       | Zeichen                        | Deutsch                              | Datentyp   | Einheit   | Beschreibung                                                              |
|----|-------------------------|-----------------|--------------------------------|--------------------------------------|------------|-----------|---------------------------------------------------------------------------|
|  0 | id                      | ptq             | ID                             | ID Messung                           | int64      | -         | Eindeutige ID der Messung                                                 |
|  1 | rope_datetime           | ls3             | $t$                            | Zeitstempel                          | object     | -         | Startzeitpunkt der Messung laut Gerät                                     |
|  2 | treatment               | series          | treatment                      | Behandlung                           | category   | -         | Art der KS: \texttt{free}, \texttt{gefa\_dynamic}, \texttt{cobra\_static} |
|  3 | release_force_target    | series          | $F_{\mathrm{release,target}}$  | Vorspannkraft-Soll                   | float64    | kN        | Geplante Vorspannkraft im Zugseil bei Release                             |
|  4 | rope_release            | ls3             | $F_{\mathrm{release}}$         | Vorspannkraft-Ist                    | float64    | kN        | Tatsächlich realisierte Vorspannkraft im Zugseil bei Release              |
|  5 | cable_max               | ls3             | $F_{\mathrm{cable, max}}$      | Kraftspitze KS                       | object     | kN        | Maximale gemessene Kraftspitze in der KS                                  |
|  6 | sensor_name             | ptq             | sensor                         | Sensorname                           | object     | -         | Bezeichnung des Elastometers                                              |
|  7 | location                | sensor_position | location                       | Position                             | string     | -         | Position des Sensors am Stamm                                             |
|  8 | direction               | sensor_position | direction                      | Richtung                             | string     | -         | Zug- oder Druckseite                                                      |
|  9 | height                  | sensor_position | $h$                            | Höhe                                 | Float64    | m         | Höhe des Sensors am Stamm                                                 |
| 10 | diameter                | sensor_position | $d$                            | Durchmesser                          | Float64    | cm        | Durchmesser des Stammes                                                   |
| 11 | max_strain              | ptq             | $\Delta L_{\mathrm{max}}$      | Dehnung max. gemessen                | float64    | $\mu$m    | Gemessene maximale absolute Randfaserdehnung                              |
| 12 | max_compression         | ptq             | $\Delta L_{\mathrm{comp,max}}$ | Stauchung max. gemessen              | float64    | $\mu$m    | Gemessene maximale absolute Randfaserstauchung                            |
| 13 | m_amplitude             | ptq_osc         | $mA$                           | Manuelle Amplitude                   | float64    | $\mu$m    | Manuell berechnete Amplitude über den Schwingungsabschnitt                |
| 14 | m_amplitude_2           | ptq_osc         | $mA_2$                         | Manuelle Amplitude 2                 | float64    | $\mu$m    | Manuell berechnete Amplitude zwischen 2. Peak und Minimum                 |
| 15 | initial_amplitude       | ptq_osc         | $A$                            | Anfangsamplitude                     | float64    | $\mu$m    | Initiale Amplitude der angepassten Schwingung                             |
| 16 | damping_coeff           | ptq_osc         | $\delta$                       | Dämpfungskoeffizient                 | float64    | 1/s       | Koeffizient der exponentiellen Dämpfung                                   |
| 17 | frequency_damped        | ptq_osc         | $f_{\mathrm{d}}$               | Gedämpfte Frequenz                   | float64    | Hz        | Frequenz der gedämpften Schwingung                                        |
| 18 | phase_angle             | ptq_osc         | $\varphi$                      | Phasenwinkel                         | float64    | rad       | Anfangsphase der Schwingung                                               |
| 19 | y_shift                 | ptq_osc         | $y_0$                          | Vertikaler Versatz                   | float64    | $\mu$m    | Vertikaler Offset der Schwingung                                          |
| 20 | x_shift                 | ptq_osc         | $t_0$                          | Zeitverschiebung                     | float64    | s         | Horizontale Verschiebung der Schwingung                                   |
| 21 | frequency_undamped      | ptq_osc         | $f_0$                          | Ungedämpfte Frequenz                 | float64    | Hz        | Frequenz der ungedämpften Schwingung                                      |
| 22 | damping_ratio           | ptq_osc         | $D$                            | Dämpfungsgrad                        | float64    | -         | Verhältnis von Dämpfung zu Frequenz                                       |
| 23 | metrics_warning         | ptq_osc         | warning                        | Fit-Warnung                          | bool       | -         | Warnung, wenn Qualitätsmetrik Schwellenwerte unterschreitet               |
| 24 | pearson_r               | ptq_osc_metric  | $r$                            | Pearson-Korrelation                  | float64    | -         | Korrelationskoeffizient der Anpassung                                     |
| 25 | nrmse                   | ptq_osc_metric  | $\mathrm{NRMSE}$               | Normalisierter RMSE                  | float64    | -         | Normalisierter mittlerer quadratischer Fehler                             |
| 26 | nmae                    | ptq_osc_metric  | $\mathrm{NMAE}$                | Normalisierter MAE                   | float64    | -         | Normalisierter mittlerer absoluter Fehler                                 |
| 27 | calc_max_strain         | calc_strain     | $\Delta L_{\mathrm{calc,max}}$ | Dehnung max. berechnet               | float64    | $\mu$m    | Berechnete maximale Randfaserdehnung                                      |
| 28 | calc_max_strain_relativ | calc_strain     | $\Delta L_{\mathrm{calc,rel}}$ | Berechnete maximale Randfaserdehnung | float64    | $\mu$m    | Differenz gemessene und berechnete maximale Randfaserdehnung              |
| 29 | strain_difference       | calc_strain     | $\Delta \varepsilon$           | Differenz der Faserdehnung           | float64    | \%        | Relative Differenz der berechneten und gemessenen max. Dehnung            |

In [17]:
df.head(10)

id       rope_datetime treatment  release_force_target  rope_release  \
4    2 2022-03-23 11:24:23      free                   2.8       2.72310   
5    2 2022-03-23 11:24:23      free                   2.8       2.72310   
6    2 2022-03-23 11:24:23      free                   2.8       2.72310   
7    2 2022-03-23 11:24:23      free                   2.8       2.72310   
8    3 2022-03-23 11:31:12      free                   2.8       2.76205   
9    3 2022-03-23 11:31:12      free                   2.8       2.76205   
10   3 2022-03-23 11:31:12      free                   2.8       2.76205   
11   3 2022-03-23 11:31:12      free                   2.8       2.76205   
12   4 2022-03-23 11:54:33      free                   2.8       2.73950   
13   4 2022-03-23 11:54:33      free                   2.8       2.73950   

    cable_max sensor_name location   direction  height  ...  frequency_damped  \
4         NaN  Elasto(90)      StB  elongation   16.55  ...          0.441518   
5         NaN  Elasto(92)      StB  elongation    11.6  ...          0.432441   
6         NaN  Elasto(95)      StA  elongation    11.6  ...          0.422702   
7         NaN  Elasto(98)      StA  elongation   16.85  ...          0.443529   
8         NaN  Elasto(90)      StB  elongation   16.55  ...          0.440653   
9         NaN  Elasto(92)      StB  elongation    11.6  ...          0.439321   
10        NaN  Elasto(95)      StA  elongation    11.6  ...          0.436023   
11        NaN  Elasto(98)      StA  elongation   16.85  ...          0.441350   
12        NaN  Elasto(90)      StB  elongation   16.55  ...          0.439065   
13        NaN  Elasto(92)      StB  elongation    11.6  ...          0.436850   

    phase_angle    y_shift   x_shift  frequency_undamped  damping_ratio  \
4      0.200000  -5.287164  0.116641            0.442936       0.503994   
5     -0.200000 -14.233644 -0.009730            0.436601       0.873595   
6     -0.200000 -19.011309 -0.123043            0.426319       0.823807   
7     -0.065967  -1.099515  0.118608            0.445993       0.663224   
8      0.200000   7.775284  0.097887            0.441317       0.344951   
9     -0.075771   7.052210  0.094719            0.441380       0.608931   
10    -0.200000  -3.785755 -0.075741            0.438127       0.618021   
11    -0.200000 -17.747289 -0.203345            0.442888       0.525070   
12     0.200000 -23.445975  0.120929            0.439764       0.354677   
13    -0.200000 -19.454720  0.061912            0.438829       0.598701   

    metrics_warning  pearson_r     nrmse      nmae  
4             False   0.943158  0.038866  0.018011  
5             False   0.929554  0.034560  0.016183  
6             False   0.908058  0.044873  0.024274  
7             False   0.980663  0.029621  0.021087  
8             False   0.957380  0.043035  0.018619  
9             False   0.933463  0.043534  0.021255  
10            False   0.938307  0.039020  0.022498  
11            False   0.974139  0.028693  0.020715  
12            False   0.945320  0.049764  0.021509  
13            False   0.922665  0.050822  0.020659  

[10 rows x 27 columns]

In [18]:
df.columns

Index(['id', 'rope_datetime', 'treatment', 'release_force_target',
       'rope_release', 'cable_max', 'sensor_name', 'location', 'direction',
       'height', 'diameter', 'max_strain', 'max_compression', 'm_amplitude',
       'm_amplitude_2', 'initial_amplitude', 'damping_coeff',
       'frequency_damped', 'phase_angle', 'y_shift', 'x_shift',
       'frequency_undamped', 'damping_ratio', 'metrics_warning', 'pearson_r',
       'nrmse', 'nmae'],
      dtype='object')

### COMBINED: Definition von Darstellungsstandards
Festlegen von Farbcodes für einheitliche Darstellung von Sensoren und Behandlungsvarianten für alle nachfolgenden Plots.

In [19]:
color_palette = PLOT_MANAGER.color_palette_list

# Für die Spalte "treatment":
treatment_color_dict = get_color_dict(df, "treatment", PLOT_MANAGER.color_palette_list)
# Für die Spalte "sensor_name":
sensor_color_dict = get_color_dict(df, "sensor_name", PLOT_MANAGER.color_palette_list)

Index(['free', 'gefa_dynamic', 'cobra_static'], dtype='object')
Index(['Elasto(90)', 'Elasto(92)', 'Elasto(95)', 'Elasto(98)'], dtype='object')


### Spitzenlasten in der KS, maximale Messung Beispieldaten

In [20]:
# Filtern der Einträge mit dem maximalen Wert in 'cable_max'
max_cable_max = df['cable_max'].max()
filtered_df = df[df['cable_max'] == max_cable_max]

# Innerhalb der gefilterten Einträge den maximalen 'max_strain' finden
max_value_row = filtered_df.loc[filtered_df['max_strain'].idxmax()]
max_value_row

id                                       22
rope_datetime           2022-03-23 16:21:50
treatment                      cobra_static
release_force_target                    2.8
rope_release                         2.7807
cable_max                              3.09
sensor_name                      Elasto(90)
location                                StB
direction                        elongation
height                                16.55
diameter                            0.18462
max_strain                            382.4
max_compression                       -67.8
m_amplitude                           224.3
m_amplitude_2                         75.55
initial_amplitude                 169.98753
damping_coeff                      0.361833
frequency_damped                   0.546096
phase_angle                       -0.074058
y_shift                            0.057425
x_shift                            0.327542
frequency_undamped                 0.549124
damping_ratio                   

In [21]:
# Liste der gewünschten Spalten in der gewünschten Reihenfolge
columns_to_display = [
    'id', 'treatment', 'release_force_target', 'rope_release', 
    'cable_max', 'sensor_name', 'location', 'height', 'diameter', 'max_strain', 'max_compression'
]

# Zeile auf die gewünschten Spalten in der angegebenen Reihenfolge beschränken
max_value_row_filtered = max_value_row[columns_to_display]

max_value_row_filtered['treatment'] = slugify(max_value_row_filtered['treatment'])

# Erstelle ein DataFrame mit den zusätzlichen Informationen aus `data_dict`
expanded_data = []
for col in columns_to_display:
    expanded_data.append({
        "Zeichen": data_dict[col]["Zeichen"],
        "Deutsch": data_dict[col]["Deutsch"],
        "Wert": max_value_row_filtered[col],
        "Einheit": data_dict[col]["Einheit"],
    })

# Neues DataFrame erstellen
expanded_df = pd.DataFrame(expanded_data)
expanded_df

Zeichen                  Deutsch          Wert  \
0                               ID               ID Messung            22   
1                        treatment               Behandlung  cobra-static   
2    $F_{\mathrm{release,target}}$       Vorspannkraft-Soll           2.8   
3           $F_{\mathrm{release}}$        Vorspannkraft-Ist        2.7807   
4        $F_{\mathrm{cable, max}}$           Kraftspitze KS          3.09   
5                           sensor               Sensorname    Elasto(90)   
6                         location                 Position           StB   
7                              $h$                     Höhe         16.55   
8                              $d$              Durchmesser       0.18462   
9        $\Delta L_{\mathrm{max}}$    Dehnung max. gemessen         382.4   
10  $\Delta L_{\mathrm{comp,max}}$  Stauchung max. gemessen         -67.8   

   Einheit  
0        -  
1        -  
2       kN  
3       kN  
4       kN  
5        -  
6        -  
7        m  
8       cm  
9   $\mu$m  
10  $\mu$m

In [22]:
# LaTeX-String erstellen
latex_string = expanded_df.to_latex(
    index=False, 
    escape=False, 
    column_format="llrr",  # Spaltenformat angepasst
    float_format="{:0.2f}".format
)

caption = "Feldversuch 2 - Ergebnisse, Werte der Messung mit Spitzenlast"
caption_long = "Feldversuch 2 - Ergebnisse, Messung mit der höchsten Kraftspitze in der KS (nur Werte des Elastometers mit max. Faserdehnung)"

save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_werte_der_messung_mit_spitzenlast.tex
